In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np 
import pandas as pd 

In [4]:
# change this later
folder = "/content/drive/MyDrive/COMP9900 Project/Machine Learning/Dataset/Books/"

# load datasets
books = pd.read_csv(folder + "books_cleaned.csv", sep=",", error_bad_lines=False, engine='python')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


# Calculate Scores (For Cold Start)

In [16]:

def get_initial_recommendations(books_df, num_recommendation = 20, count_threshold_percent = 0.1):

  # average ratings
  overall_ratings = books_df.average_rating.mean()

  # as the dataset is large, we only care about the top n% most rated books
  min_count = books_df.ratings_count.quantile(1 - count_threshold_percent)
  popular_books = books_df.copy().loc[books_df["ratings_count"] >= min_count]

  # compute the score, taking account both rating and the number of ratings
  def weighted_rating(df, m = min_count, C = overall_ratings):
      v = df['ratings_count']
      R = df['average_rating']
      # calculation based true Bayesian estimate used in the IMDB formula
      score = (v / (v + m) * R) + (m / (m + v) * C)
      return score

  # calculate the score and sort the results based on the score
  popular_books['score'] = popular_books.apply(weighted_rating, axis=1)
  popular_books = popular_books.sort_values('score', ascending=False)

  # top books we should recommend for the new users with zero interaction 
  recommended_books = popular_books[['book_id', 'title', 'authors', 'ratings_count', 'average_rating', 'score']].head(num_recommendation)

  # comment/uncomment this to debug and evaluate
  print(recommended_books[['title', 'ratings_count', 'average_rating', 'score']])

  # returns the recommended book ID
  return list(recommended_books['book_id'])

In [17]:
print(get_initial_recommendations(books, 20, 0.1))

                                                 title  ratings_count  \
24   Harry Potter and the Deathly Hallows (Harry Po...        1746574   
26   Harry Potter and the Half-Blood Prince (Harry ...        1678823   
17   Harry Potter and the Prisoner of Azkaban (Harr...        1832823   
23   Harry Potter and the Goblet of Fire (Harry Pot...        1753043   
397           Harry Potter Boxset (Harry Potter, #1-7)         190050   
129     A Storm of Swords (A Song of Ice and Fire, #3)         469022   
20   Harry Potter and the Order of the Phoenix (Har...        1735368   
183  The Name of the Wind (The Kingkiller Chronicle...         400101   
1    Harry Potter and the Sorcerer's Stone (Harry P...        4602479   
30                                            The Help        1531753   
38      A Game of Thrones (A Song of Ice and Fire, #1)        1319204   
154  The Return of the King (The Lord of the Rings,...         463959   
288  The Wise Man's Fear (The Kingkiller Chronicle,